In [ ]:
### This code makes a dataset with 
# y = (2016 solar additions in MW)/(solar MW capacity built < 2016)
# x = (net metering, RPS, RPS w/Solar, avg. retail electricity price, solar index, regulated/not, population, other 2016 cap additions, community solar, region)


In [66]:
import pandas as pd
import datetime
import numpy as np
import statsmodels.formula.api as sm
import statsmodels.api as smap
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)

### Clean Price Data

In [67]:
# Code cleans up data for prices (monthly, state)

data = pd.ExcelFile('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/monthly_prices_sales_revenue.xlsx')

df = data.parse(sheetname=data.sheet_names[0], skiprows=0)

# make a list of the header row and strip up to the 4th letter. This is the location and year information
cols1 = list(df.columns)
cols1 = [str(x).lower() for x in cols1]

# make another list of the first row,this is the age group information
# we need to preserve this information in the column name when we reshape the data 
cols2 = list(df.iloc[0,:])
cols2 = [str(x).lower() for x in cols2]

cols3 = list(df.iloc[1,:])
cols3 = [str(x).lower().replace(" ","_") for x in cols3]

# now join the two lists to make a combined column name which preserves our location, year and age-group information
cols = [x+"_"+y for x,y in zip(cols1,cols2)]
name = ""
for i,x in enumerate(cols):
    if "unnamed" in x:
        x = x.split('_')[-1]
        cols[i] = x
    if "_" in x:
        name = x.split("_")[0]
        continue
    elif ("_" not in x):
        cols[i] = name+"_" + x
cols[0:4] = cols3[0:4]
# Assign new column names to the dataframe
df.columns = cols
df.drop([0,1], inplace=True)
df = df[:-1]
df.to_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/avg_price_monthly.csv')

In [70]:
# Import electricity price data (annual)

price_data = pd.read_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/avg_price_monthly.csv')

price_data['date'] = pd.to_datetime([f'{y}-{m}-01' for y, m in zip(price_data.year, price_data.month)])

### Clean generation data

In [69]:
# Import generating data (annual)

data = pd.ExcelFile('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/eia_summary_data/annual_generation_state.xls')

ann_generation = data.parse(sheetname=data.sheet_names[0], skiprows=1)
ann_generation.columns = [x.lower().replace(" ","_") for x in ann_generation.columns]
tot_gen = ann_generation[ann_generation['type_of_producer'].str.contains('Total Electric Power')]

### Clean capacity data

In [24]:
# Import annual capacity data

cap_data = pd.read_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/eia_capacity_data/existcapacity_annual.csv', header=1)
cap_data.columns = [x.lower().replace(" ","_") for x in cap_data.columns]
cap_data['summer_cap']=pd.to_numeric(cap_data['summer_capacity_(megawatts)'].str.replace(",",""),errors='coerce')
cap_data['nameplate_cap']=pd.to_numeric(cap_data['nameplate_capacity_(megawatts)'].str.replace(",",""),errors='coerce')

In [33]:
solar_cap = cap_data[(cap_data['producer_type']=="Total Electric Power Industry") &
                     (cap_data['fuel_source'].str.contains("Solar"))]

In [58]:
solar_gen = tot_gen[tot_gen['energy_source']== "Solar Thermal and Photovoltaic"]

In [63]:
reg1_data = solar_cap.merge(solar_gen, left_on=['year','state_code'],right_on=['year','state'],how='inner')
reg1_data.to_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/StataDatasets/cap_gen.csv')

### Clean generator dataset

In [442]:
# Import generator data (survey of all generators in 2018)

gen_data = pd.read_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/eia_data/eia8602018/merged_new.csv')
gen_data = gen_data.drop(gen_data.columns[gen_data.columns.str.contains("Unnamed")], axis=1)
gen_data['operating_year'] = gen_data['operating_year'].astype(int)
gen_data['operating_month'] = gen_data['operating_month'].astype(int)
gen_data['operating_date'] = pd.to_datetime([f'{y}-{m}-01' for y, m in zip(gen_data.operating_year, gen_data.operating_month)])
gen_data['summer_cap']=pd.to_numeric(gen_data['summer_capacity_mw'].str.replace(",","").replace(" ",""))
gen_data['winter_cap']=pd.to_numeric(gen_data['winter_capacity_mw'].str.replace(",","").replace(" ",""))

/Users/rachelanderson/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (18,41,45,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [443]:
solar = gen_data[gen_data['technology'] == "Solar Photovoltaic"]
solar = solar[solar['grant_program']=='none']

In [457]:
solar_cap_monthly = solar.groupby(['plant_state','operating_date','operating_month','operating_year'])[['summer_cap','nameplate_cap', 'winter_cap']].sum().reset_index()
res_price_monthly = price_data[['date', 'state', 'residential_price']]

In [460]:
monthly_vars = solar_cap_monthly.merge(res_price_monthly, how='right', left_on = ['operating_date', 'plant_state'], right_on=['date','state'])
monthly_vars = monthly_vars.drop(columns=['plant_state','operating_date'], axis = 1)
monthly_vars=monthly_vars.fillna(0)

In [461]:
monthly_vars.to_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/monthly_data.csv')

In [434]:
grant_data = pd.read_excel('../../../Data for Tax Equity Project/treasury_data/grant_awards.xlsx',header=1)

In [466]:
loan_funds = grant_data.groupby('State').Funded.sum().reset_index()

In [472]:
solar_cap_ann = solar.groupby(['plant_state','operating_year'])[['summer_cap','nameplate_cap', 'winter_cap']].sum().reset_index()

In [471]:
# y = 2016 capacity addition as a percent of stock up until 2016

,plant_state,operating_year,summer_cap,nameplate_cap,winter_cap
0,AL,2016,75.0,75.0,75.0
1,AL,2017,104.4,107.2,101.4
2,AL,2018,14.7,14.7,14.7
3,AR,2015,12.0,12.0,12.0
4,AR,2016,1.0,1.0,1.0
...,...,...,...,...,...
266,WI,2013,1.0,1.0,1.0
267,WI,2016,2.1,2.1,2.1
268,WI,2017,15.3,15.3,15.3
269,WI,2018,3.7,3.7,3.7
